In [ ]:
!pip install fonttools brotli

In [ ]:
import os

SCRIPT_DIR = os.getcwd()

base_name = os.path.splitext(os.path.basename(__file__))[0]
file_type = os.path.splitext(os.path.basename(__file__))[1]
print(base_name)
print(file_type)

In [ ]:
from fontTools.ttLib import TTFont

def convert_font_to_web_formats(input_font_path, output_dir="."):
    """
    Converts a TTF or OTF font to WOFF and WOFF2 formats.
    With the wisdom of Epictetus, we shall not waste effort on that which already exists.

    Args:
        input_font_path (str): The path to the input font file (.ttf or .otf).
        output_dir (str): The directory where the converted fonts will be saved.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Get base name and normalize it for web use
    base_name = os.path.splitext(os.path.basename(input_font_path))[0]
    # Normalize to web-friendly naming: lowercase, remove all non-alphanumeric characters
    web_safe_name = ''.join(c.lower() if c.isalnum() else '' for c in base_name)
    
    # Define output paths
    woff_output_path = os.path.join(output_dir, f"{web_safe_name}.woff")
    woff2_output_path = os.path.join(output_dir, f"{web_safe_name}.woff2")
    
    # Check if both formats already exist - if so, skip entirely
    if os.path.exists(woff_output_path) and os.path.exists(woff2_output_path):
        return False  # Nothing was processed
    
    # Load font once for both conversions
    try:
        font = TTFont(input_font_path)
    except Exception as e:
        print(f"Error loading font {input_font_path}: {e}")
        return False
    
    # Convert to WOFF if needed
    if not os.path.exists(woff_output_path):
        try:
            font.flavor = "woff"
            font.save(woff_output_path)
        except Exception as e:
            print(f"Error converting {input_font_path} to WOFF: {e}")

    # Convert to WOFF2 if needed (reload font to avoid flavor conflicts)
    if not os.path.exists(woff2_output_path):
        try:
            font = TTFont(input_font_path)  # Reload to clear previous flavor
            font.flavor = "woff2"
            font.save(woff2_output_path)
        except Exception as e:
            print(f"Error converting {input_font_path} to WOFF2: {e}")
    
    return True  # Something was processed

def process_fonts_directory(source_dir, output_base_dir):
    """
    Walks through the entire directory structure, finds all TTF and OTF files,
    and converts them to optimized web formats while preserving the directory structure.
    
    Verily, as Epictetus teaches us that what matters is not what happens to us but how we respond,
    so too must we approach this task with methodical discipline and unwavering purpose.
    Each font shall be processed but once, for efficiency is the highest virtue.

    Args:
        source_dir (str): The root directory containing font folders
        output_base_dir (str): The base directory where converted fonts will be saved
    """
    
    if not os.path.exists(source_dir):
        print(f"Source directory does not exist: {source_dir}")
        return
    
    # Create the base output directory if it doesn't exist
    if not os.path.exists(output_base_dir):
        os.makedirs(output_base_dir)
    
    font_extensions = ('.ttf', '.otf', '.TTF', '.OTF')
    processed_count = 0
    skipped_count = 0
    processed_files = set()  # Track already processed files to prevent duplicates
    
    print(f"Beginning the font conversion from: {source_dir}")
    print(f"Output will be written to: {output_base_dir}")
    print("-" * 80)
    
    # Walk through all directories and subdirectories
    for root, dirs, files in os.walk(source_dir):
        # Get the relative path from the source directory and normalize for web
        rel_path = os.path.relpath(root, source_dir)
        
        # Normalize directory names to web-friendly format
        if rel_path != ".":
            # Split path components, normalize each consistently, and rejoin
            path_parts = rel_path.split(os.sep)
            # Use same normalization as file names: lowercase alphanumeric only
            normalized_parts = [''.join(c.lower() if c.isalnum() else '' for c in part) for part in path_parts]
            normalized_rel_path = os.path.join(*normalized_parts)
            output_dir = os.path.join(output_base_dir, normalized_rel_path)
        else:
            output_dir = output_base_dir
        
        # Find all font files in the current directory
        font_files = [f for f in files if f.lower().endswith(font_extensions)]        
        
        if font_files:
            # print(f"\nProcessing directory: {root}")
            # print(f"Found {len(font_files)} font file(s)")
            
            # Ensure output directory exists
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            
            # Process each font file
            for font_file in font_files:
                input_font_path = os.path.join(root, font_file)
                
                # Create unique identifier to prevent duplicate processing
                file_identifier = os.path.abspath(input_font_path)
                if file_identifier in processed_files:
                    continue  # Skip if already processed
                
                base_name = os.path.splitext(font_file)[0]
                # Normalize filename for web use - same as directory normalization
                web_safe_name = ''.join(c.lower() if c.isalnum() else '' for c in base_name)
                
                # Check if both WOFF and WOFF2 already exist
                woff_exists = os.path.exists(os.path.join(output_dir, f"{web_safe_name}.woff"))
                woff2_exists = os.path.exists(os.path.join(output_dir, f"{web_safe_name}.woff2"))
                
                if woff_exists and woff2_exists:
                    # print(f"  Skipping {font_file} - both formats already exist")
                    skipped_count += 1
                else:
                    # print(f"  Converting: {font_file}")
                    try:
                        if convert_font_to_web_formats(input_font_path, output_dir):
                            processed_count += 1
                    except Exception as e:
                        print(f"  Error processing {font_file}: {e}")
                
                # Mark this file as processed regardless of outcome
                processed_files.add(file_identifier)
    
    print("Task completed.\n" + "-" * 80)
    print(f"- Processed {processed_count} font files\n- Skipped {skipped_count} already converted.")

if __name__ == "__main__":
    # Define the source and output directories
    source_fonts_dir = r"C:\Users\Your\Downloads\Path\To\Fonts"
    output_fonts_dir = SCRIPT_DIR  # Use current script directory as output base
    
    # Execute the font processing with philosophical determination
    process_fonts_directory(source_fonts_dir, output_fonts_dir)
    
    print("\nAll fonts have been converted to WOFF and WOFF2 formats.")